In [1]:
import torch
import torch.nn as nn
import os
import cv2
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
import warnings
from tqdm import tqdm
warnings.filterwarnings('ignore')
from torchvision import models
from torchvision import transforms
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix

In [2]:
if(os.path.split(os.getcwd())[1] == "classify-to-learn"):
    os.chdir("..")
print("Current Working Directory: {}".format(os.path.split(os.getcwd())[1]))

cuda = False

Current Working Directory: progettoVIPM


In [3]:
from utils.loadersAndEnums import datasets
from utils.loadersAndEnums import networks
from utils.loadersAndEnums import ImageDataset
from utils.v2 import ExtendedEncoder, Encoder

In [4]:
from utils.loadersAndEnums import datasets, ImageDataset
from torch.utils.data import DataLoader, ConcatDataset, random_split

trainU = ImageDataset(dataset=datasets.TRAINING_UNLABELED,network_input_size=224, cuda=cuda)
unlabeled = DataLoader(trainU,128,True)

In [5]:

device = torch.device('cpu')
#PATH = "Storage\models\AutoEncoderModelV2.pth"
PATH = 'Storage\models\\finetunedResNet50.pth'

if cuda:
    model = torch.load(PATH)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
else:    
    model = torch.load(PATH, map_location=torch.device('cpu') )


In [84]:
all_preds = []
all_out = []
model.eval()
with torch.no_grad():
    for inputs,labels in tqdm(unlabeled):
        labels = [int(label) for label in labels]
        inputs, labels = inputs.to(device), torch.tensor(labels, dtype=torch.long).to(device)
        outputs = model(inputs)
        print('outputs: ', outputs)
        print('outputs: ', outputs.shape)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_out.extend(outputs.cpu().numpy())
        
all_preds = np.array(all_preds)
all_out = np.array(all_out)


  0%|          | 1/887 [00:11<2:47:42, 11.36s/it]

outputs:  tensor([[-1.6666, -1.9691, -1.0686,  ..., -4.5402, -3.1432, -2.9515],
        [-3.1942, -3.2076, -3.1805,  ..., -3.7901, -2.7987, -2.1621],
        [-1.9498, -1.0083, -2.7844,  ..., -5.4915, -3.2666, -3.9290],
        ...,
        [-4.1192, -3.3378, -2.9261,  ..., -2.9806, -2.2687, -2.1883],
        [-4.4205, -5.0510, -4.6419,  ..., -2.1618, -4.2665, -5.1066],
        [-2.9069, -3.3245, -2.1991,  ..., -3.6803, -3.3446, -1.8926]])
outputs:  torch.Size([128, 251])


  0%|          | 1/887 [00:12<3:02:46, 12.38s/it]


KeyboardInterrupt: 

In [45]:
all_preds.tofile('all_preds.dat')

In [46]:
all_out.tofile('all_out.dat')

In [87]:
all_outfromfile = np.fromfile('all_out.dat', dtype=int)
print('all_outfromfile.shape: ', all_outfromfile.shape)
print('all_outfromfile: ', all_outfromfile[251*10:251*12])

all_predsfromfile = np.fromfile('all_preds.dat', dtype=int)
print('all_predsfromfile: ', all_predsfromfile)

all_outfromfile.shape:  (28477205,)
all_outfromfile:  [-1070931824 -1069267536 -1069650263 -1070620610 -1071884314 -1070480964
 -1079860997 -1069214571 -1082602092 -1072695682 -1068751722 -1069631141
 -1069555971 -1072689109 -1069098055 -1073414665  1060149054 -1073634578
 -1072362353 -1071214926 -1069418709 -1077015618 -1067007476  1042777096
 -1070476069 -1073492754 -1069249537 -1072697994 -1066728322 -1071095806
 -1075406935 -1068272512 -1092062016 -1072978807 -1070920509 -1069578744
  1067450245 -1076712856  1039174530 -1063895380 -1075559034  1062882524
 -1067030334 -1074177763 -1067499476 -1072756067 -1068811684 -1071270038
 -1070046202 -1066275754 -1071792042 -1068558380 -1071019690 -1069350033
 -1105428555 -1071235272 -1070479974 -1073783774 -1071958758 -1073666373
 -1067825344  1058991777 -1065844614 -1072550017 -1070210528 -1069769060
 -1067933593 -1073134292 -1065255725 -1070648944  1065580004 -1083152274
 -1071063306 -1071230975 -1070256007  1083035977 -1071259582 -10689329

In [85]:
#outputs = torch.from_numpy(all_out[251:251*2])
print('outputs: ', outputs[1])
#print('outputs: ', outputs.shape)
m, predicted = torch.max(outputs, 1)
print('predicted: ', predicted)
print('m: ', m)
all_out=[]
all_out.extend(outputs.cpu().numpy())
all_out = np.array(all_out)
print('all_out: ', all_out)

outputs:  tensor([-3.1942, -3.2076, -3.1805, -1.5229, -5.3111, -3.6859, -3.4493, -1.6922,
        -4.3194, -4.5440, -2.2851, -0.4768, -2.9174, -1.0429, -3.1577, -4.8870,
        -5.0210, -0.8064, -3.2109, -4.6794, -2.8105, -4.8863, -2.8291, -4.4676,
        -2.7008, -3.0495, -3.8444, -2.7970, -3.4362, -3.0425, -2.9252, -2.7762,
        -4.3907, -2.5772, -1.7246, -4.1167, -3.2755, -0.6474, -3.8971, -2.5141,
        -3.1906, -2.9875, -3.6658, -5.1596, -4.3877, -1.6885, -3.6453, -2.8201,
        -3.5226, -3.7529, -4.2427, -4.4593, -3.1527, -0.8813, -3.6429, -2.2493,
        -3.2671,  1.6185, -2.7247, -3.9661, -0.5604, -2.3728, -3.8313, -1.3625,
        -4.8442, -0.0520, -2.7110, -3.1933, -5.9032, -2.7781, -3.8988, -4.1031,
        -2.3454, -4.1806, -3.1994, -5.2528, -4.0405, -2.3643, -3.8101, -1.1431,
        -1.8954, -3.0582, -3.2240, -2.4123, -4.6173, -2.4923, -3.7934, -4.5722,
        -4.2167, -4.7777, -1.5372, -2.4505, -1.3454, -3.4602, -2.3712,  0.2293,
        -1.4169, -2.1820, -2.9